In [1]:
from correctingagent.pddl.goal_updates import *
from correctingagent.agents.PGMAgent import read_sentence
from correctingagent.models.pgmmodels import evaluate_rule

In [2]:
neg_rule =create_negative_goal(['red'], ['blue'])

In [3]:
read_sentence("no, don't put red blocks on blue blocks", use_dmrs=False)

Message(rel='on', o1=['red'], o2=['blue'], T='neg', o3=None)

In [9]:
rule = create_neg_rule('red', 'blue')

In [5]:
evaluate_rule('red', 1, 'blue', 1, rule)

'Undefined'

In [6]:
rule

'- (exists x. exists y. red(x) and blue(y) and on(x,y))'

In [19]:
rule = '- exists x. exists y. (red(x) and blue(y) and on(x,y))'
evaluate_rule('red', 0, 'blue', 1, rule)

True

In [17]:
evaluate_rule('red', 1, 'blue', 0, rule)

'Undefined'

In [18]:
rule = 'all x. all y. - (red(x) and blue(y) and on(x,y))'
print(evaluate_rule('red', 0, 'blue', 0, rule))
print(evaluate_rule('red', 1, 'blue', 0, rule))
print(evaluate_rule('red', 0, 'blue', 1, rule))
print(evaluate_rule('red', 1, 'blue', 1, rule))

True
True
True
False


In [21]:
rule = '- exists x. exists y. (red(x) and blue(y) and on(x,y))'
print(evaluate_rule('red', 0, 'blue', 0, rule))
print(evaluate_rule('red', 1, 'blue', 0, rule))
print(evaluate_rule('red', 0, 'blue', 1, rule))
print(evaluate_rule('red', 1, 'blue', 1, rule))

True
True
True
False


In [23]:
neg_rule

'(not (exists (?x ?y) (and (red ?x) (blue ?y) (on ?x ?y))))'

In [24]:
def get_relevant_colours(rule):
    if rule.op == 'not':
        exists = neg_rule.subformulas[0]
        and_ = exists.subformulas[0]
        c1, c2, on = and_.subformulas
        return get_name(c1), get_name(c2), 'not'
    else:
    
        or_ = rule.subformulas[0]
        r1, r2 = or_.subformulas
        colour1 = (r1.subformulas[0])
        colour2 = (list(filter(lambda x: get_name(x) != 'on', r2.subformulas[0].subformulas))[0])
        on = list(filter(lambda x: get_name(x) == 'on', r2.subformulas[0].subformulas))[0]
        o1, o2 = on.get_predicates(True)[0].args.args
        out = (o1, o2)
        c1 = list(filter(lambda x: get_args(x)[0] == o1.arg_name, [colour1, colour2]))[0]
        c2 = list(filter(lambda x: get_args(x)[0] == o2.arg_name, [colour1, colour2]))[0]
        return get_name(c1), get_name(c2), get_name(colour1) # on(c1, c2) colour1 -> colour2

In [28]:
get_relevant_colours(neg_rule)

ValueError: not enough values to unpack (expected 2, got 1)

In [30]:
neg_rule.op

'not'

In [31]:
neg_rule.asPDDL()

'(not (exists (?x ?y) (and (red ?x) (blue ?y) (on ?x ?y))))'

In [44]:
exists = neg_rule.subformulas[0]
and_ = exists.subformulas[0]
c1, c2, on = and_.subformulas

In [55]:
c1.get_predicates(True)[0].name

'red'

In [14]:
beam = {'r':1, 'b':0, 'c':1, 'g':1}
worlds = [{'r':1}, {'b':0}] #True 
worlds2 = [{'r':1}, {'b':1}] #True
worlds3 = [{'r':1}, {'b':0, 'c':1}] # True
worlds4 = [{'r':0}, {'b':1}] # False
worlds5 = [{'r':0}, {'b':0, 'c':1}] # True
worlds6 = [{'r':0}, {'b':1, 'c':1}] # False
worlds7 = [{'r':0}] # False
worlds8 = []
worlds9 = [{'q':1}]
worlds10 = [{'q':1}, {'r':1}]
def check_beam_holds(beam, worlds):
    if not worlds:
        return True
    return any([all([beam[var] == w[var] if var in beam.keys() else False for var in w.keys()]) for w in worlds])

print(check_beam_holds(beam, worlds))
print(check_beam_holds(beam, worlds2))
print(check_beam_holds(beam, worlds3))
print(check_beam_holds(beam, worlds4))
print(check_beam_holds(beam, worlds5))
print(check_beam_holds(beam, worlds6))
print(check_beam_holds(beam, worlds7))
print(check_beam_holds(beam, worlds8))
print(check_beam_holds(beam, worlds9))
print(check_beam_holds(beam, worlds10))

True
True
True
False
True
False
False
True
False
True
